# Initial setup

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import json
#from google.colab.patches import cv2_imshow
from six.moves import urllib
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [3]:
#from tensorflow.keras.applications.resnet50 import preprocess_input
from keras import applications
preprocess_input = applications.mobilenet_v2.preprocess_input 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

Unzip data-.zip

In [ ]:
! unzip ~/Documents/Dataset/external.zip -d ~/Documents/Dataset
! unzip ~/Documents/Dataset/persone_daclassi_test.zip -d ~/Documents/Dataset

In [ ]:
! zip -r persone_da_classiBN.zip ~/Documents/Dataset/persone_da_classiBN

In [ ]:
! unzip ~/Documents/Dataset/trainBN600.zip -d ~/Documents/Dataset

Folder named *data-* must contain folders with target name and synset labels with images inside, as follows:

<pre>
<b>data-</b>
|__ <b>Persona</b>
|__ <b>n01443537</b>
|__ <b>n01532829</b>
|__ <b>n--------</b>
|__ <b>n--------</b>
</pre>

A second folder named *train-* is created to host *target* folder with  people images inside and *reference* folder with images belonging to reference classes inside, as follows:

<pre>
<b>train-</b>
|__ <b>target</b>
    |__ <b>Persona</b>
|__ <b>reference</b>
    |__ <b>African elephant, Loxodonta africana</b>
    |__ <b>acoustic guitar</b>
    |__ <b>analog clock</b>
    |__ <b>backpack, back pack, knapsack, packsack, rucksack, haversack</b>
    |__ <b>beer glass</b>
</pre>

Set useful paths:

In [5]:
path_ds = "Dataset"
path_data = os.path.join(path_ds, "data")  #Dataset/data2
#path_train = os.path.join(path_ds, "trainBN")
path_train = os.path.join(path_ds, "trainBN200")
if not os.path.exists(path_train):
      os.makedirs(path_train)
target = "Persona"

path_reference = os.path.join(path_train, "reference")  #Dataset/train2/reference
if not os.path.exists(path_reference):
      os.makedirs(path_reference)
path_target = os.path.join(path_train, "target")        #Dataset/train2/target
if not os.path.exists(path_target):
      os.makedirs(path_target)

In [6]:
print("There are ", len(os.listdir(os.path.join(path_data, target))), " images from Target dataset")

There are  6000  images from Target dataset


In [7]:
n_classes_ref = len(os.listdir(path_data))-1
print("There are " + str(n_classes_ref) + " classes belonging to reference dataset")

There are 20 classes belonging to reference dataset


# Training part


## Pre-process images BN (do it only if new images are added)

Create the synset_to_human dictionary to map synsets to human-readable names for every synset in Imagenet. This helps in creating folder structure shown before.

In [ ]:
def create_readable_names_for_imagenet_labels():
    """Create a dict mapping label id to human readable string.
    Returns:
            synset_to_human: dictionary mapping synset and human-readable names.

    'imagenet_lsvrc_2015_synsets.txt' contains a list of valid synset labels used by ILSVRC competition.
    There is one synset one per line, eg.
              #   n01440764
              #   n01443537
    'imagenet_metadata.txt' contains a mapping from synsets to human-readable names for every synset in Imagenet. 
    These are stored in a tsv format, as follows:
              #   n02119247    black fox
              #   n02119359    silver fox

    Code is based on
    https://github.com/tensorflow/models/blob/master/inception/inception/data/build_imagenet_data.py#L463
    """
    filename = 'imagenet_lsvrc_2015_synsets.txt'
    synset_list = [s.strip() for s in open(filename).readlines()]
    num_synsets_in_ilsvrc = len(synset_list)
    assert num_synsets_in_ilsvrc == 1000

    filename = 'imagenet_metadata.txt'
    synset_to_human_list = open(filename).readlines()
    num_synsets_in_all_imagenet = len(synset_to_human_list)
    assert num_synsets_in_all_imagenet == 21842

    synset_to_human = {}
    for s in synset_to_human_list:
        parts = s.strip().split('\t')
        assert len(parts) == 2
        synset = parts[0]
        human = parts[1]
        if synset in synset_list:
            synset_to_human[synset] = human

    return synset_to_human

synset_to_human = create_readable_names_for_imagenet_labels()

In [ ]:
print(len(synset_to_human))

Pre-processing BN

In [ ]:
img_size = 224
for folder in os.listdir(path_data):
    path_folder = os.path.join(path_data, folder)
    print("\n------------------------------------------------------")
    print("\nFolder ", folder, " with ", len(os.listdir(path_folder)), "images inside")

    if folder == target:
        path_out = os.path.join(path_target, folder)
        if not os.path.exists(path_out):
            os.makedirs(path_out)
    else: 
        path_out = os.path.join(path_reference, synset_to_human[folder])
        if not os.path.exists(path_out):
            os.makedirs(path_out)

    i=0   #new images
    j=0   #images already pre-processed
    for file in os.listdir(path_folder):
        if os.path.exists(path_out + "/" + file):
            j+=1
            print("Image " + file + " already pre-processed" )
        else:
            i+=1
            print("Processing ... ", file)
            
            #read the image
            image = cv2.imread(path_folder + "/"+ file)
            #crop image -> square image along its min dimension
            h, w, c = image.shape
            if w>h:
                start = (w-h)//2
                image = image[:, start:start+h]
            else:
                start = (h-w)//2
                image = image[start:start+w,:]
            #resize
            image = cv2.resize(image, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     #gray image
            image = cv2.merge((image, image, image))            #gray image on 3 channels
            #write the pre-proc image in train folder
            cv2.imwrite(path_out + "/" + file, image)

    print("\nImages that have been previously pre-processed: " + str(j))
    print("\nNewly pre-processed images: " + str(i))

Find which classes are part of the reference dataset:

In [8]:
classes_ref = []  #list
classes_ref = [name for name in os.listdir(path_reference)]
print("Classes of reference dataset are ", classes_ref)

Classes of reference dataset are  ['beer glass', 'African elephant, Loxodonta africana', 'ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle', 'jaguar, panther, Panthera onca, Felis onca', 'koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus', 'monarch, monarch butterfly, milkweed butterfly, Danaus plexippus', 'pitcher, ewer', 'rocking chair, rocker', 'backpack, back pack, knapsack, packsack, rucksack, haversack', 'analog clock', 'passenger car, coach, carriage', 'mosque', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'candle, taper, wax light', 'acoustic guitar', 'space bar', 'squirrel monkey, Saimiri sciureus', 'house finch, linnet, Carpodacus mexicanus', 'pot, flowerpot', 'goldfish, Carassius auratus']


## Create model


MobileNetV2

In [9]:
classes = len(os.listdir(path_data))-1
alpha = 1.0 #for MobileNetV2
base_model = applications.MobileNetV2(include_top=False, 
                                      input_shape=(224, 224, 3), 
                                      alpha=alpha, 
                                      weights='imagenet',
                                      pooling="avg")

In [10]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [11]:
predictions = Dense(classes, activation='softmax')(base_model.output)
model = keras.Model(inputs=base_model.input,outputs=[base_model.get_layer("global_average_pooling2d").output, predictions])

In [ ]:
#Seconda versione con 
#FC1 = Dense(540, activation='relu')(base_model.output) #540 neurons because we don't want sharp variation from 1280 to n_classes_ref
#FC2 = Dense(200, activation='relu')(FC1)              #we can add other layers later
#predictions = Dense(n_classes_ref, activation = 'softmax')(FC2)

#model = keras.Model(inputs=base_model.input,outputs=[base_model.get_layer("global_average_pooling2d").output, predictions])

In [ ]:
#We extract features from average pooling layer to take advantages from loaded pre-trained weights. FC1 output is not pre-trained!
#model = keras.Model(
#    inputs=base_model.input,
#    #outputs= [base_model.get_layer("avg_pool").output, predictions]   
#    outputs= [base_model.get_layer("global_average_pooling2d").output, predictions]
#)

In [12]:
#n_features = base_model.get_layer("avg_pool").output.shape[1]   #2048
n_features = base_model.get_layer("global_average_pooling2d").output.shape[1]   #1280

print(n_features)

1280


In [ ]:
print(model.output) #shape=(None, 1280) shape=(None, 20)

In [ ]:
model.summary()

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

In [13]:
#Fixed weights
for layer in model.layers:
    #if layer.name == "Conv_1":
    if layer.name == "block_13_expand":
        break
    else:
        layer.trainable = False

In [ ]:
#model.trainable = True

In [ ]:
#Layers not frozen are: output layer, fully connected layer, 2 convolutional layers  + their batch normalization layers
#for layer in model.layers[:len(model.layers)-10]:
#    layer.trainable = False

In [14]:
k=0
for layer in model.layers:
    #print(layer, layer.trainable)
    if layer.trainable == True:
        k=k+1
print("Layers with trainable=True: ", k, "")

Layers with trainable=True:  40 


In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

## Losses and compile

In [15]:
batch_size = 32
sub_batch_size = batch_size // 2
beta = sub_batch_size**2 / (sub_batch_size-1)**2  #1.0158 with batch_size=256
print("beta = ", beta)

beta =  1.1377777777777778


In [16]:
#Compactness loss is computed using intermediate features of average pooling layer. Assuming n_features = 2048:
#used for X_target, first half of output -> [0:128]
#y_true has shape (256, n_classes_ref), created by inputgenerator. No problems in dimension mis-match since y_true is not used here.
#y_pred is feature_batch[0] of shape (128, 2048)

def compactness_loss(y_true, y_pred):
    #y_pred_target = y_pred[0:128]   #shape (128, 2048)
    y_pred_target = y_pred[0:16]   #shape (16, 2048)
    # ERRATA -> l_c = tf.keras.backend.mean(tf.keras.backend.var(y_pred_target, axis = 1, keepdims=False)) 
    #axis = 1 means variance along the row -> tf.keras.backend.var of shape=(128,)
    l_c = tf.keras.backend.mean(tf.keras.backend.var(y_pred_target, axis = 0, keepdims=False)) 
    #axis = 0 means variance along the columns (so the features)-> tf.keras.backend.var of shape=(2048,)

    return l_c * beta

#when features are extraxted from convolutional layer: apply average pooling layer ->  compute loss 

In [17]:
#Categorical crossentropy loss used in the descriptiveness loss
cce = tf.keras.losses.CategoricalCrossentropy(from_logits=False) 

#**Note - Using from_logits=True is more numerically stable.** -> remove softmax layer
#used default redution: reduction=losses_utils.ReductionV2.AUTO

In [18]:
#used for X_reference, second half of output -> [128:256]
#y_true has shape (256, n_classes_ref), created by inputgenerator = concatenation of Y_target_one_hot (128, n_classes_ref) and Y_ref_one_hot (128, n_classes_ref)
#y_pred is feature_batch[1] of shape (256, n_classes_ref)

def descriptiveness_loss(y_true, y_pred): 
    #y_true_reference = y_true[128:256]  #shape (128, n_classes_ref)
    #y_pred_reference = y_pred[128:256]  #shape (128, n_classes_ref)
    y_true_reference = y_true[16:32]  #shape (16, n_classes_ref)
    y_pred_reference = y_pred[16:32]  #shape (16, n_classes_ref)
    l_d = cce(y_true_reference, y_pred_reference)
    return l_d

In [19]:
lambd = 10

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00005, decay=0.00005), #, momentum=0.9),  #lr=0.2, momentum=0.9, decay=0.01
    #optimizer=tf.keras.optimizers.Adam( learning_rate=0.00005,
    #                                    beta_1=0.9,
    #                                    beta_2=0.999,
    #                                    epsilon=1e-07,
    #                                    amsgrad=False,
    #                                    name="Adam"),
    loss=[compactness_loss, descriptiveness_loss],
    #loss={"FC1": compactness_loss,
    #      "predictions": descriptiveness_loss},
    
    loss_weights = [lambd, 1],
    metrics=["accuracy"],
)

## Create multiple image generator for training



In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [20]:
#output of inputgenerator is a tuple : ((batch_size, 224, 224, 3), (batch_size, 20))

input_imgen = ImageDataGenerator(preprocessing_function = preprocess_input)

def generate_generator_multiple(generator, dir1, dir2, sub_batch_size, img_height, img_width, n_classes):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = sub_batch_size,
                                          shuffle=True)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = sub_batch_size,
                                          shuffle=True)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield np.concatenate([X1i[0], X2i[0]]), np.concatenate([to_categorical(np.argmax(X1i[1], axis=1), num_classes=n_classes_ref), X2i[1]]) 
            #Yield 2 concatenated batches and their categorical concatenated labels
            
inputgenerator = generate_generator_multiple(generator = input_imgen,
                                             dir1 = path_target,
                                             dir2 = path_reference,
                                             sub_batch_size = sub_batch_size,
                                             img_height = 224,
                                             img_width = 224,
                                             n_classes = n_classes_ref)       

In [21]:
train_size = len(os.listdir(os.path.join(path_target, target))) #6000
epochs = 400

In [22]:
print(train_size)

200


## Train with *fit*

In [ ]:
history = model.fit(inputgenerator,
                    epochs = epochs,
                    steps_per_epoch = train_size // sub_batch_size,
                    #use_multiprocessing=True,
                    #shuffle=False
                    )

In [ ]:
model.metrics_names

Retrive losses and accuracy from history

In [ ]:
# Retrieve losses and accuracy
total_loss = history.history['loss']
l_c = history.history['avg_pool_loss']
l_d = history.history['dense_1_loss']
acc_dense = history.history['dense_1_accuracy']


# Get number of epochs
epochs = range(len(total_loss))

print("Total loss = ", total_loss)
print("Compactness loss = ", l_c)
print("Descriptiveness loss = ", l_d)
print("Accuracy (dense) = ", acc_dense)

## Train with *train_on_batch*

In [ ]:
total_loss=total_loss
l_c=l_c
l_d=l_d

In [23]:
total_loss=[]
l_c=[]
l_d=[]
acc=[]

In [ ]:
n_batches = train_size // sub_batch_size
print("Number of batches : ", n_batches)

for epoch in range(epochs):
    print("\nEpoch ", epoch+1 , "/", epochs)
  
    for i in range(n_batches):
        print("Processing batch...  ", i)
        batch = next(inputgenerator)
        #print(type(batch), batch[0].shape, batch[1].shape)
        loss, compactness_loss, descriptness_loss, dense_accuracy, dense_1_accuracy = model.train_on_batch(batch[0], batch[1])
        #Print the total loss every 10 iterations
        if i % 10 == 0:
            print("\nTotal loss after iteration ", i, " is ", loss)
            print("\nCompact loss after iteration ", i, " is ", compactness_loss)
            print("\nDescript loss after iteration ", i, " is ", descriptness_loss)
            total_loss.append(loss)
            l_c.append(compactness_loss)
            l_d.append(descriptness_loss)
            acc.append(dense_1_accuracy)
            
    if (epoch+1) % 50 == 0:
        my_model = "my_model_l400_"+ str(epoch+1) +".h5"
        path_model = os.path.join(path_ds, my_model)  #/content/drive/My Drive/my_model.h5
        model.save(path_model)

    print("Total loss at the end of epoch " , epoch+1, ": ", loss)

Number of batches :  12

Epoch  1 / 400
Processing batch...   0
Found 200 images belonging to 1 classes.
Found 10000 images belonging to 20 classes.

Total loss after iteration  0  is  6.188575267791748

Compact loss after iteration  0  is  0.31376463174819946

Descript loss after iteration  0  is  3.050929069519043
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  6.489594459533691

Compact loss after iteration  10  is  0.3342227637767792

Descript loss after iteration  10  is  3.147367000579834
Processing batch...   11
Total loss at the end of epoch  1 :  5.869030952453613

Epoch  2 / 400
Processing batch...   0

Total loss after iteration  0  is  6.316370010375977

Compact loss after iteration  0  is  0.29484546184539795

Descript loss after iteration  0  is 

Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  6.108537197113037

Compact loss after iteration  10  is  0.30274587869644165

Descript loss after iteration  10  is  3.08107852935791
Processing batch...   11
Total loss at the end of epoch  12 :  6.041297435760498

Epoch  13 / 400
Processing batch...   0

Total loss after iteration  0  is  6.314696311950684

Compact loss after iteration  0  is  0.30827417969703674

Descript loss after iteration  0  is  3.231954574584961
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  6.27866268157959

Compact loss after iteration  10  is  0.31072112917900085

Descript loss after iteration  10  is  3.1714510917663574
Processing batch...   11
Total loss at the end of epoch  13 :  5.58609437942

Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  5.782114028930664

Compact loss after iteration  10  is  0.30427590012550354

Descript loss after iteration  10  is  2.7393548488616943
Processing batch...   11
Total loss at the end of epoch  24 :  5.696646690368652

Epoch  25 / 400
Processing batch...   0

Total loss after iteration  0  is  5.436671257019043

Compact loss after iteration  0  is  0.2856864631175995

Descript loss after iteration  0  is  2.5798065662384033
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  6.109945774078369

Compact loss after iteration  10  is  0.30275532603263855

Descript lo


Total loss after iteration  0  is  5.475673675537109

Compact loss after iteration  0  is  0.2704200744628906

Descript loss after iteration  0  is  2.771472930908203
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  5.1178741455078125

Compact loss after iteration  10  is  0.27152907848358154

Descript loss after iteration  10  is  2.402583122253418
Processing batch...   11
Total loss at the end of epoch  36 :  5.499619960784912

Epoch  37 / 400
Processing batch...   0

Total loss after iteration  0  is  5.138915538787842

Compact loss after iteration  0  is  0.27527013421058655

Descript loss after iteration  0  is  2.386214256286621
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Process

Total loss at the end of epoch  47 :  5.384056091308594

Epoch  48 / 400
Processing batch...   0

Total loss after iteration  0  is  5.334385871887207

Compact loss after iteration  0  is  0.2917885184288025

Descript loss after iteration  0  is  2.4165005683898926
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  5.5630903244018555

Compact loss after iteration  10  is  0.2810377776622772

Descript loss after iteration  10  is  2.7527122497558594
Processing batch...   11
Total loss at the end of epoch  48 :  5.623868942260742

Epoch  49 / 400
Processing batch...   0

Total loss after iteration  0  is  6.090113639831543

Compact loss after iteration  0  is  0.3084864914417267

Descript loss after iteration  0  is  3.0052490234375
Processing batch...   1
Processi

...

Total loss at the end of epoch  272 :  1.8602674007415771

Epoch  273 / 400
Processing batch...   0

Total loss after iteration  0  is  2.2757058143615723

Compact loss after iteration  0  is  0.09711233526468277

Descript loss after iteration  0  is  1.3045825958251953
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  2.0027706623077393

Compact loss after iteration  10  is  0.08556536585092545

Descript loss after iteration  10  is  1.147117018699646
Processing batch...   11
Total loss at the end of epoch  273 :  3.185262680053711

Epoch  274 / 400
Processing batch...   0

Total loss after iteration  0  is  2.1782779693603516

Compact loss after iteration  0  is  0.08864713460206985

Descript loss after iteration  0  is  1.291806697845459
Processing batch...  

Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.6297650337219238

Compact loss after iteration  10  is  0.07344383746385574

Descript loss after iteration  10  is  0.8953266143798828
Processing batch...   11
Total loss at the end of epoch  284 :  1.748004674911499

Epoch  285 / 400
Processing batch...   0

Total loss after iteration  0  is  1.9569675922393799

Compact loss after iteration  0  is  0.09481014311313629

Descript loss after iteration  0  is  1.0088661909103394
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  2.868901491165161

Compact loss after iteration  10  is  0.14106659591197968

Descript loss after iteration  10  is  1.458235740661621
Processing batch...  

Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.7920911312103271

Compact loss after iteration  10  is  0.08637558668851852

Descript loss after iteration  10  is  0.9283351898193359
Processing batch...   11
Total loss at the end of epoch  296 :  2.0829684734344482

Epoch  297 / 400
Processing batch...   0

Total loss after iteration  0  is  2.1077394485473633

Compact loss after iteration  0  is  0.09018556773662567

Descript loss after iteration  0  is  1.2058837413787842
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.7264575958251953

Compact loss after iteration  10  is  0

Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.587517261505127

Compact loss after iteration  10  is  0.07660749554634094

Descript loss after iteration  10  is  0.8214423656463623
Processing batch...   11
Total loss at the end of epoch  308 :  2.261187791824341

Epoch  309 / 400
Processing batch...   0

Total loss after iteration  0  is  1.9528651237487793

Compact loss after iteration  0  is  0.08156247437000275

Descript loss after iteration  0  is  1.1372404098510742
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.9716310501098633

Compact loss after iteration  10  is  0.0

Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.7403349876403809

Compact loss after iteration  10  is  0.07917309552431107

Descript loss after iteration  10  is  0.9486039876937866
Processing batch...   11
Total loss at the end of epoch  320 :  2.091346502304077

Epoch  321 / 400
Processing batch...   0

Total loss after iteration  0  is  2.245530605316162

Compact loss after iteration  0  is  0.07830363512039185

Descript loss after iteration  0  is  1.4624942541122437
Processing batch...   1
Processing batch...   2
Processing batch...   3
Processing batch...   4
Processing batch...   5
Processing batch...   6
Processing batch...   7
Processing batch...   8
Processing batch...   9
Processing batch...   10

Total loss after iteration  10  is  1.669128656387329

Compact loss after

## Retrieve losses stored in drive

In [ ]:
path_loss = os.path.join(path_drive, "loss19.json") #/content/drive/My Drive/loss.json
path_lc = os.path.join(path_drive, "l_c19.json")
path_ld = os.path.join(path_drive, "l_d19.json")

In [ ]:
with open(path_loss, 'r') as fp:
  total_loss = json.load(fp)
with open(path_lc, 'r') as fp:
  l_c = json.load(fp)
with open(path_ld, 'r') as fp:
  l_d = json.load(fp)

## Plot losses

In [ ]:
print(total_loss)
print(l_c)
print(l_d)

In [ ]:
print(np.mean(total_loss))
print(np.mean(l_c))
print(np.mean(l_d))

In [ ]:
len(l_c)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_loss, label="Total loss")
plt.plot(l_c, label="Compacteness loss")
plt.plot(l_d, label="Descriptiveness loss")
plt.xlabel("training steps")
plt.legend()
plt.show()

In [ ]:
plt.plot(total_loss, label="Total loss")
plt.xlabel("training steps")
plt.legend()
#plt.xscale('log')
plt.show()

In [ ]:
plt.plot(l_c, label="Compacteness loss")
plt.xlabel("training steps")
#plt.xscale('log')
plt.legend()
plt.show()

In [ ]:
plt.plot(l_d, label="Descriptiveness loss")
plt.xlabel("training steps")
#plt.xscale('log')
plt.legend()
plt.show()

In [ ]:
plt.plot(acc, label="Dense accuracy")
plt.xlabel("training steps")
#plt.xscale('log')
plt.legend()
plt.show()

## Save losses on Drive

In [ ]:
path_loss = os.path.join(path_drive, "loss20.json") #/content/drive/My Drive/loss.json
path_lc = os.path.join(path_drive, "l_c20.json")
path_ld = os.path.join(path_drive, "l_d20.json")

In [ ]:
with open(path_loss, 'w') as fp:
  json.dump(total_loss, fp)
with open(path_lc, 'w') as fp:
  json.dump(l_c, fp)
with open(path_ld, 'w') as fp:
  json.dump(l_d, fp)

## Save model

In [ ]:
path_model = os.path.join(path_ds, "my_model_400.h5")  #/content/drive/My Drive/my_model.h5
model.save(path_model)  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# Testing part

## Load model from Drive and obtain *model_features* for feature extraction

In [ ]:
path_model = os.path.join(path_ds, "my_model_300.h5")  #/content/drive/My Drive/my_model.h5

In [ ]:
model_tot = load_model(path_model, custom_objects={'compactness_loss': compactness_loss, 'descriptiveness_loss': descriptiveness_loss})

In [ ]:
model = load_model(path_model, custom_objects={'compactness_loss': compactness_loss, 'descriptiveness_loss': descriptiveness_loss})

In [ ]:
model_tot.summary()

In [ ]:
model.summary()

In [ ]:
# AttributeError: 'ListWrapper' object has no attribute 'name' -> the last layer 
#in my model._layers is a ListWrapper, because I am using multiple loss functions

#from tensorflow.keras.layers import Layer
#model_tot._layers = [
#    layer for layer in model_tot._layers if isinstance(layer, Layer)
#]
#keras.utils.plot_model(model_tot, "model_tot.png", show_shapes=True)

From *model_tot* extrapolate the model for feature extraction : *model_features*

In [ ]:
#model_features = Model(model_tot.inputs, model_tot.layers[-3].output) #output = <tf.Tensor 'avg_pool_1/Identity:0' shape=(None, 2048) dtype=float32>

In [ ]:
model_features = Model(model_tot.inputs, model_tot.layers[-2].output) #output = <tf.Tensor 'dense_2/Identity:0' shape=(None, 1024) dtype=float32>

In [ ]:
#model_features = Model(model.inputs, model.layers[-3].output) #output = <tf.Tensor 'avg_pool_1/Identity:0' shape=(None, 2048) dtype=float32>

In [ ]:
model_features = Model(model.inputs, model.layers[-2].output) #output = <tf.Tensor 'avg_pool_1/Identity:0' shape=(None, 2048) dtype=float32>

In [ ]:
model_features.summary()

## Templates generation BN

In [ ]:
path_images_templates = os.path.join(path_ds, "templates")  #/content/drive/My Drive/templates

In [ ]:
path_images_templatesBN = os.path.join(path_ds, "templatesBN")  #/content/drive/My Drive/templates

In [ ]:
img_size = 224
for folder in os.listdir(path_images_templates):
    path_folder = os.path.join(path_images_templates, folder)
    print("\n------------------------------------------------------")
    print("\nFolder ", folder, " with ", len(os.listdir(path_folder)), "images inside")

    path_out = os.path.join(path_images_templatesBN, "templates_Persona")
    if not os.path.exists(path_out):
        os.makedirs(path_out)

    i=0   #new images
    j=0   #images already pre-processed
    for file in os.listdir(path_folder):
        if os.path.exists(path_out + "/" + file):
            j+=1
            print("Image " + file + " already pre-processed" )
        else:
            i+=1
            print("Processing ... ", file)
            
            #read the image
            image = cv2.imread(path_folder + "/"+ file)
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     #gray image
            image = cv2.merge((image, image, image))            #gray image on 3 channels
            #write the pre-proc image in train folder
            cv2.imwrite(path_out + "/" + file, image)

    print("\nImages that have been previously pre-processed: " + str(j))
    print("\nNewly pre-processed images: " + str(i))
  

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(path_images_templatesBN,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= len(os.listdir(os.path.join(path_images_templatesBN, "templates_Persona"))))  #50

templates = model_features.predict(test_generator,steps = 1)  #(n_templates, n_features) ex. (40, 2048)

In [ ]:
from matplotlib import pyplot as plt
batch = next(test_generator)
for i in range (0,40):
    image = batch[0][i]
    plt.imshow(image)
    plt.show()

In [ ]:
templates.shape

## Save templates in Drive

In [ ]:
path_templates = os.path.join(path_ds, "templates.npy") 
np.save(path_templates, templates, allow_pickle=True, fix_imports=True)

## Retrive templates if present on Drive


In [ ]:
path_templates = os.path.join(path_drive, "templates3.npy") 
if os.path.exists(path_templates):
  templates = np.load(path_templates)

In [ ]:
templates.shape

## Create  features_test and Y_test (with labels 0: target class =Person, 1: alien class, no people inside)

Folder on Drive named *data_test* contains folders with target name and Others, as follows:

<pre>
<b>data_test</b>
|__ <b>Persona</b>
|__ <b>Others</b>
</pre>

It contains images to be tested

Images will be processed and saved in *test* folder with the following structure:
<pre>
<b>test</b>
|__ <b>Persona</b>
   |__ <b>1</b>
|__ <b>Others</b>
   |__ <b>0</b>

</pre>


## Pre-process test images

*path_test* contains pre-processed BN images

In [ ]:
path_data_test = os.path.join(path_ds, "data_test") 

path_test = os.path.join(path_ds, "testBN")

path_test_Persona = os.path.join(path_test, "Persona")

path_test_Others = os.path.join(path_test, "Others")

Pre-process test images

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
img_size = 224
for folder in os.listdir(path_data_test):
    path_folder = os.path.join(path_data_test, folder)
    print("\n------------------------------------------------------")
    print("\nFolder ", folder, " with ", len(os.listdir(path_folder)), "images inside")

    if folder == target:
        path_out = os.path.join(path_test_Persona, "0")
        if not os.path.exists(path_out):
            os.makedirs(path_out)
    else: 
        path_out = os.path.join(path_test_Others, "1")
        if not os.path.exists(path_out):
            os.makedirs(path_out)

    i=0   #new images
    j=0   #images already pre-processed
    for file in os.listdir(path_folder):
        if os.path.exists(path_out + "/" + file):
            j+=1
            print("Image " + file + " already pre-processed" )
        else:
            i+=1
            print("Processing ... ", file)
            
            #read the image
            image = cv2.imread(path_folder + "/"+ file)
            #crop image -> square image along its min dimension
            h, w, c = image.shape
            if w>h:
                start = (w-h)//2
                image = image[:, start:start+h]
            else:
                start = (h-w)//2
                image = image[start:start+w,:]
            #resize
            image = cv2.resize(image, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     #gray image
            image = cv2.merge((image, image, image))            #gray image on 3 channels
            #write the pre-proc image in train folder
            cv2.imwrite(path_out + "/" + file, image)

    print("\nImages that have been previously pre-processed: " + str(j))
    print("\nNewly pre-processed images: " + str(i))
  

## Extract features from *X_test* obtaining *features_X_test*

Extract features from Person images

In [ ]:
test_datagen1 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator1 = test_datagen1.flow_from_directory(path_test_Persona,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 100
                                                  ) 

In [ ]:
features_Persona = model_features.predict(test_generator1, steps = len(os.listdir(os.path.join(path_test_Persona, "0"))) // 100)

In [ ]:
features_Persona.shape

Extract features from Others images

In [ ]:
test_datagen0 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator0 = test_datagen0.flow_from_directory(path_test_Others,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 100
                                                  ) 

In [ ]:
features_Others = model_features.predict(test_generator0, steps = len(os.listdir(os.path.join(path_test_Others, "1"))) // 100)

In [ ]:
features_Others.shape

In [ ]:
features_test = np.concatenate([features_Persona, features_Others])    #Y_test will be 00000 ... 11111

In [ ]:
features_test_tsne = np.concatenate([features_Persona, features_Others, templates]) 

In [ ]:
features_test.shape

In [ ]:
Y_test=np.concatenate([np.zeros(features_Persona.shape[0]), np.ones(features_Others.shape[0])])

In [ ]:
Y_test_tsne=np.concatenate([np.zeros(features_Persona.shape[0]), np.ones(features_Others.shape[0]), np.ones(templates.shape[0])*2])

In [ ]:
Y_test.shape

Including y_pred for external & pers_daclassi

In [ ]:
path_externalBN = os.path.join(path_ds, "externalBN")
path_pers_da_classiBN = os.path.join(path_ds, "persone_da_classiBN")

!rm -rf `find -type d -name .ipynb_checkpoints`
test_datagen1 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator1 = test_datagen1.flow_from_directory(path_externalBN,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 60
                                                  ) 

y_pred_external = model_features.predict(test_generator1, steps = 1)

y_pred_external.shape

!rm -rf `find -type d -name .ipynb_checkpoints`
test_datagen1 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator1 = test_datagen1.flow_from_directory(path_pers_da_classiBN,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 60
                                                  ) 

y_pred_persdaclassi = model_features.predict(test_generator1, steps = 1)

In [ ]:
features_test = np.concatenate([features_Persona, features_Others, y_pred_external, y_pred_persdaclassi])

Y_test=np.concatenate([np.zeros(features_Persona.shape[0]), np.ones(features_Others.shape[0]), np.ones(y_pred_external.shape[0]), np.zeros(y_pred_persdaclassi.shape[0])])

In [ ]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

feat_cols = [ 'pixel'+str(i) for i in range(features_test.shape[1]) ]
df = pd.DataFrame(features_test,columns=feat_cols)
df['y'] = Y_test

#feat_cols = [ 'pixel'+str(i) for i in range(features_test_tsne.shape[1]) ]
#df = pd.DataFrame(features_test_tsne,columns=feat_cols)
#df['y'] = Y_test_tsne

df['label'] = df['y'].apply(lambda i: str(i))
#features_test, Y_test = None, None
print('Size of the dataframe: {}'.format(df.shape))

# For reproducability of the results
np.random.seed(4)
rndperm = np.random.permutation(df.shape[0])

N = Y_test.shape[0]
df_subset = df.loc[rndperm[:N],:].copy()
data_subset = df_subset[feat_cols].values
#pca = PCA(n_components=3)
#pca_result = pca.fit_transform(data_subset)
#df_subset['pca-one'] = pca_result[:,0]
#df_subset['pca-two'] = pca_result[:,1] 
#df_subset['pca-three'] = pca_result[:,2]
#print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=df_subset,
    legend="full",
    alpha=1
)

## Save *features_X_test* on Drive

In [ ]:
path_X_test_features= os.path.join(path_drive, "features_X_test10.npy") 
np.save(path_X_test_features, features_X_test, allow_pickle=True, fix_imports=True)

## Retrive features_X_test if present on Drive


In [ ]:
features_X_test = np.load(path_X_test_features)

In [ ]:
features_X_test.shape

## Template matching

In [ ]:
#def euclidean(v1, v2):
#  return sum((p-q)**2 for p, q in zip(v1, v2)) ** .5

#d = [euclidean(f, t) for t in templates]


In [ ]:
def scores_generation(features_test, templates):
    for f in features_test:
        d = [np.linalg.norm(f-t) for t in templates]  #np.linalg.norm(f-t) = Euclidean norm 
        distances_vector.append(d)                    #shape of (n_X_test, n_templates): euclidean norm between each element of features_X_test and each template  
    scores = np.amin(distances_vector, axis=1)      #axis=1 -> min value for each row
    #scores = np.mean(distances_vector, axis=1)
    scores = np.array(scores)
    return scores

In [ ]:
scores = []
distances_vector = []

In [ ]:
scores = scores_generation(features_test, templates)  #shape of (n_X_test,) for each image belonging to X_test

In [ ]:
scores.shape

In [ ]:
scores

In [ ]:
scores

## Plot ROC curve

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
#from sklearn.metrics import roc_auc_score

In [ ]:
def plot_roc_curve(fpr, tpr, label):
  plt.plot(fpr, tpr, linewidth=2, label=label)
  plt.legend()
  plt.title('ROC curve')
  plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.grid(True)

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, scores)
AUC = roc_auc_score(Y_test, scores)
print(AUC)

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, scores)
AUC = roc_auc_score(Y_test, scores)
print(AUC)

In [ ]:
plot_roc_curve(fpr, tpr, label='DeepOneClassification(AUC = %.2f)'%AUC)
plt.show()

Optimal threshold

In [ ]:
thresholds

In [ ]:
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)

In [ ]:
y_pred = np.zeros(scores.shape[0])
for i in range(scores.shape[0]):
    if scores[i] > optimal_threshold:
        y_pred[i] = 1

In [ ]:
np.sum(y_pred[0:1000])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)
#           predited                  0:negative - persona   1:positive - others
           #   0  1
#actual    #0 TN FP                   FN=false negative, actual others but predicted person
           #1 FN TP                   FP=false positive, actual person but predicted others

F1score

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y_test, y_pred)

Variance Person features

In [ ]:
features_Persona = tf.convert_to_tensor(features_Persona, np.float32)

In [ ]:
tf.keras.backend.var(features_Persona, axis = 0, keepdims=False)

In [ ]:
var = tf.keras.backend.mean(tf.keras.backend.var(features_Persona, axis = 0, keepdims=False))
print(var)

## New approach for testing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

#train = train.reshape((len(X_train_s),-1))
#test_s = test_s.reshape((len(X_test_s),-1))
#test_b = test_b.reshape((len(X_test_b),-1))

#Convert to 0-1
ms = MinMaxScaler()
train = ms.fit_transform(templates)
test_s = ms.transform(features_test)


In [ ]:
# fit the model
clf = LocalOutlierFactor(n_neighbors=5)
y_pred = clf.fit(train)

#Abnormal score
Z1 = -clf._decision_function(test_s[0:500])
Z2 = -clf._decision_function(test_s[500:1000])

#Drawing of ROC curve
y_true = np.zeros(len(test_s[0:500])+len(test_s[500:1000]))
y_true[len(test_s[0:500]):] = 1 #0:Normal, 1：Abnormal

#Calculate FPR, TPR(, Threshould)
fpr, tpr, _ = metrics.roc_curve(y_true, np.hstack((Z1, Z2)))

#AUC
auc = metrics.auc(fpr, tpr)

#Plot the ROC curve
plt.plot(fpr, tpr, label='DeepOneClassification(AUC = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)
plt.show()